# Intro
Here, we train a XGB model over different hparams/configs

In [1]:
from kaggle_ubiquant.dataset import generate_dataset, DatasetConfig, Dataset
from kaggle_ubiquant.model import generate_model, ModelConfig
from kaggle_ubiquant.train import training_run
from definitions import ROOT_DIR
import pandas as pd
import numpy as np
from typing import Iterable, Dict, Tuple, Callable, Optional
from scipy.stats import pearsonr
import wandb
import dataclasses

/Users/dennis/repos/kaggle-ubiquant/.venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df = pd.read_csv(ROOT_DIR / 'data/train_smallest.csv')
df.head()

,Unnamed: 0,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
1,1,0_35,0,35,-0.201764,0.370986,0.615937,-0.619879,1.381941,-0.221554,...,-0.121848,0.912726,1.496875,0.819155,-1.060166,0.260081,-1.087009,-1.490426,-0.797171,0.010625
2,2,0_47,0,47,-0.113607,0.153591,-1.016361,1.561905,-0.592690,-0.572665,...,-0.170365,0.912726,-1.060477,-1.220772,0.941183,-0.650731,1.296864,1.338143,1.102991,-0.726626
3,3,0_51,0,51,0.149378,0.055568,-0.388554,0.274646,-0.600506,-0.504672,...,-0.170365,0.912726,-0.777580,-1.220772,0.941183,-0.529121,0.104928,1.139248,-0.439243,-0.594110
4,4,0_62,0,62,-0.268107,-0.158161,0.239253,1.115570,2.378658,3.569081,...,-0.105676,-1.095620,2.803366,0.819155,-1.060166,3.152009,1.296864,-1.277611,-2.236596,6.585410


# Build training run
uses wandb

In [3]:
len(pd.unique(df.investment_id))

10

In [4]:
dc = DatasetConfig(7, 5, 5)
mc = ModelConfig()

In [5]:
# Moved code to git repo

# def training_run(
#     raw_df: pd.DataFrame,
#     dataset_config: DatasetConfig, 
#     model_config: ModelConfig, 
#     wandb_project: Optional[str] = None,
# ) -> float:
#     """
#     Returns a score (goal is to maximize). In this case, it's the Pearson correlation coeff.
#     """
#     dataset = generate_dataset(dataset_config, raw_df)
#     model = generate_model(model_config)
    
#     if wandb_project:
#         wandb.init(project=wandb_project)
#         wandb.config.dataset_config = dataclasses.asdict(dataset_config)
#         wandb.config.model_config = dataclasses.asdict(model_config)
    
#     feature_columns = [elem for elem in list(dataset.train.columns) if elem != 'target']
#     X = dataset.train.loc[:, feature_columns]
#     y = dataset.train.target
#     model.fit(X, y)
    
#     preds = model.predict(dataset.test.loc[:, feature_columns])
#     targets = dataset.test.target
#     r, _ = pearsonr(preds, targets)
    
#     if wandb_project:
#         wandb.log({'pearsonr': r})
#         wandb.finish()
    
#     return r

In [6]:
training_run(df, dc, mc, 'kaggleUbiquant-scratch')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1408/1408 [00:00<00:00, 26613.22it/s]
wandb: Currently logged in as: dennisfeng (use `wandb login --relogin` to force relogin)


/Users/dennis/repos/kaggle-ubiquant/.venv/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/dennis/repos/kaggle-ubiquant/.venv/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


pearsonr,▁
pearsonr,0.0932


0.09320343653963722

# Investigate wandb.sklearn
Conclusion: only helps with visualizations/plotting, not much else

In [62]:
X_train = dataset.train.loc[:, feature_columns]
y_train = dataset.train.target
X_test = dataset.test.loc[:, feature_columns]
y_test = dataset.test.target

In [65]:
wandb.init(project='kaggleUbiquant-scratch')
wandb.sklearn.plot_regressor(model, X_train, X_test, y_train, y_test)

wandb: 
wandb: Plotting Regressor.
wandb: Logged summary metrics.
wandb: Logged learning curve.
wandb: WARNING using only the first 1000 datapoints to create chart outlier_candidates
wandb: Logged outlier candidates.
wandb: WARNING using only the first 100 datapoints to create chart residuals
wandb: WARNING using only the first 100 datapoints to create chart residuals
wandb: Logged residuals.


In [ ]:
wandb.finish()

# Build training loop
with optuna to choose hparams